<a href="https://colab.research.google.com/github/Hos96/Decision-Tree-from-scratch/blob/main/mushroom_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
from graphviz import Digraph
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# connection to my drive
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Professor Cesa Bianchi/secondary_data.csv')


Mounted at /content/drive


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                             Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                                             --------------  ----- 
 0   class;cap-diameter;cap-shape;cap-surface;cap-color;does-bruise-or-bleed;gill-attachment;gill-spacing;gill-color;stem-height;stem-width;stem-root;stem-surface;stem-color;veil-type;veil-color;has-ring;ring-type;spore-print-color;habitat;season  61069 non-null  object
dtypes: object(1)
memory usage: 477.2+ KB
None


In [ ]:

class TreeNode:
    def __init__(obj, feature=None, threshold=None, left=None, right=None, value=None):
        obj.feature = feature
        obj.threshold = threshold
        obj.left = left
        obj.right = right
        obj.value = value  # Value like p or e

    def is_leaf(obj):
        return obj.value is not None

class DecisionTree:
    def __init__(obj, maximum_depth=None, max_leaf_nodes=None, entropy_threshold=None, split_function=None, minimum_samples_split=2, feature_names=None):
        obj.maximum_depth = maximum_depth
        obj.max_leaf_nodes = max_leaf_nodes
        obj.entropy_threshold = entropy_threshold
        obj.split_function = split_function
        obj.minimum_samples_split = minimum_samples_split
        obj.root = None
        obj.feature_names = feature_names
        obj.leaf_count = 0
        obj.depth = 0
        #criterion_func is a dictionaty to map these names to the corresponding splitting functions
        obj.criterion_func = {
            'scaled_entropy': obj._scaled_entropy,
            'gini': obj._gini_impurity,
            'squared': obj._squared_impurity,
            #'misclassification': obj._misclassification
        }.get(obj.split_function)

    def get_params(obj, deep=True):
        return {
            'maximum_depth': obj.maximum_depth,
            'max_leaf_nodes': obj.max_leaf_nodes,
            'entropy_threshold': obj.entropy_threshold,
            'split_function': obj.split_function,
            'minimum_samples_split': obj.minimum_samples_split,
            'feature_names': obj.feature_names
        }

    def set_params(obj, **params):# params is a dictionary
        for param, value in params.items():
            setattr(obj, param, value)# for modifing the hyper parameters of tree
        return obj

    def _grow_tree(obj, X, y, current_depth=0):
        #X numpy array 2 dimensional, y = one dimension numpy array, current_depth at the begining is zero
        num_samples, num_features = X.shape

        current_entropy = obj.criterion_func(y)

        if current_depth > obj.depth: # for updating the depth
            obj.depth = current_depth

        if (current_depth >= obj.maximum_depth) \ #checks the below lines for stopping the tree
                or (obj.leaf_count >= obj.max_leaf_nodes) \
                or (obj.entropy_threshold is not None and current_entropy < obj.entropy_threshold) \
                or (num_samples < obj.minimum_samples_split) \
                or (np.unique(y).size == 1): #same class check
            leaf_value = obj._most_common_label(y) #assigning the class lable
            return TreeNode(value=leaf_value)

        feature_indexes = np.random.choice(num_features, num_features, replace=False)# randomness (needs to change!!)
        best_feat, best_thresh = obj._best_feature_to_split_based_on(X, y, feature_indexes)
        if best_feat is None:
            leaf_value = obj._most_common_label(y)
            return TreeNode(value=leaf_value)

        obj.leaf_count += 1
        left_idxs, right_idxs = obj._split(X[:, best_feat], best_thresh)
        left = obj._grow_tree(X[left_idxs, :], y[left_idxs], current_depth + 1)
        right = obj._grow_tree(X[right_idxs, :], y[right_idxs], current_depth + 1)
        return TreeNode(best_feat, best_thresh, left, right)

    def fit(obj, X, y):
        obj.root = obj._grow_tree(X, y)

    def _best_feature_to_split_based_on(obj, X, y, feature_indexes):
        best_information_gain = -1
        split_idx, split_thresh = None, None #split_idx: index of the best feature
        for index_counter in feature_indexes:# search for all indexes in the list of indexes
            X_column = X[:, index_counter]
            thresholds = np.unique(X_column)# split_thresh the optimal thresh for the best feature
            for threshold in thresholds:
                gain = obj._information_gain(y, X_column, threshold)
                if gain > best_information_gain:
                    best_information_gain = gain
                    split_idx = index_counter
                    split_thresh = threshold
        return split_idx, split_thresh

    def _split(obj, X_column, split_thresh): # returns two lists of left indexes(lower than thresh) and right (higher than threrhs)
        #X_column is a list containing all the values of the feature column
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _information_gain(obj, y, X_column, split_thresh):

        parent_criterion = obj.criterion_func(y)

        left_idxs, right_idxs = obj._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        y_left, y_right = y[left_idxs], y[right_idxs]
        child_criterion = obj._weighted_criterion(y_left, y_right, obj.criterion_func)
        gain = parent_criterion - child_criterion
        return gain

    def _weighted_criterion(obj, y_left, y_right, criterion_func): #Entropy/ gini average of left and right
        n = len(y_left) + len(y_right)
        p_left = len(y_left) / n
        p_right = len(y_right) / n
        return p_left * criterion_func(y_left) + p_right * criterion_func(y_right)

    # From class lectures scaled_ent = - (p/2)*np.log2(p) - ((1-p)/2)*np.log2(1-p) for binary classification
    def _scaled_entropy(obj, y):
        hist = np.bincount(y)
        probs = hist / len(y)
        scaled_ent = -np.sum([(p / 2) * np.log2(p) for p in probs if p > 0])
        return scaled_ent

    #From class lectures it should be 2p(1-p) for binary classification, but let use the general formula for non-binary case
    def _gini_impurity(obj, y):
        hist = np.bincount(y)
        probs = hist / len(y)
        gini = 1.0 - np.sum(probs ** 2)  # gini = np.sum(probs * (1 - probs))
        return gini

    # for binary classification    sqrt(p*(1-p))
    def _squared_impurity(obj, y):
        hist = np.bincount(y)
        probs = hist / len(y)
        epsilon = 1e-10  # Small constant to avoid multiplying by zero
        sqr = np.sum(np.sqrt((probs + epsilon) * (1 - probs + epsilon)))
        return sqr

    #def _misclassification(obj, y):
    #    hist = np.bincount(y)
    #    probs = hist / len(y)
    #    mce = 1.0 - np.max(probs)
    #    return mce

    def _most_common_label(obj, y):
        return np.bincount(y).argmax()

    def predict(obj, X):
        return np.array([obj.navigation_tree_for_prediction(x, obj.root) for x in X])

    def navigation_tree_for_prediction(obj, x, node):# recursive function
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return obj.navigation_tree_for_prediction(x, node.left)
        return obj.navigation_tree_for_prediction(x, node.right)

    def visualize_tree(obj, dot=None):
        if dot is None:
            dot = Digraph()

        # Recursive function to add nodes and edges
        def add_nodes_edges(dot, node): # for graphical design
            if node is None:
                return
            if node.is_leaf():
                dot.node(str(id(node)), f"Class {node.value}", shape='ellipse', style='filled', fillcolor='lightgreen')
            else:
                if obj.feature_names is not None:
                    feature_name = obj.feature_names[node.feature]
                else:
                    feature_name = f"Feature {node.feature}"
                dot.node(str(id(node)), f"{feature_name} <= {node.threshold:.2f}", shape='box', style='filled', fillcolor='lightblue')
                if node.left is not None:
                    add_nodes_edges(dot, node.left)
                    dot.edge(str(id(node)), str(id(node.left)), '<=', color='blue')
                if node.right is not None:
                    add_nodes_edges(dot, node.right)
                    dot.edge(str(id(node)), str(id(node.right)), '>', color='red')

        add_nodes_edges(dot, obj.root)
        return dot

def zero_one_loss(y_actual, y_training_prediction):
    return np.mean(y_training_prediction != y_actual)

if __name__ == '__main__':
    # Load the dataset
    data = df

    print(data.head())

    # one-hot encoding
    data_encoded = pd.get_dummies(data)
    data_encoded = data_encoded.astype(int)
    print(data_encoded.head())

    # features vs target
    X = data_encoded.drop(['class_p', 'class_e'], axis=1)  # Assuming 'class_p' is the target
    y = data_encoded['class_p']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


    tree = DecisionTree(
        maximum_depth=None,
        max_leaf_nodes=None,
        entropy_threshold=0.2,
        split_function="gini",
        minimum_samples_split=2,
        feature_names=X_train.columns
    )

    start_time = time.time()
    tree.fit(X_train.values, y_train.values)
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"\nExecution time: {execution_time} seconds")

    # predictions on the training data
    y_training_prediction = tree.predict(X_train.values)

    # model on the training data
    accuracy = accuracy_score(y_train, y_training_prediction)
    print(f"\nTraining accuracy: {accuracy:.6f}")

    # Predict on the testing data
    y_test_pred = tree.predict(X_test.values)

    # Evaluate the model on the testing data
    accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test accuracy: {accuracy:.6f}")

    train_error = zero_one_loss(y_test.values, y_test_pred)# the zero-one loss
    print(f"zero one loss on test set: {train_error:.6f}")

    #confusion matrix
    conf_matrix = confusion_matrix(y_test.values, y_test_pred)

    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Red', xticklabels=['Class 0', 'Class 1'],# show the confusion matrix
                yticklabels=['Class 0', 'Class 1'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    print("\n\n")

    # Visualize the tree
    dot = Digraph()
    dot = tree.visualize_tree(dot)
    dot.render('png/mushroom_tree', format='png', view=True)